In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate


pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

## Summary

#### Results
1. Best performed model is NN with roc_auc_score of 0.8965 on Chris's and my features.
2. NN model is saved as 'best_model2.h5'.
3. My hypothesis on synthetic drugs doesn't work. I can probably try PCA to improve the 65_brands features(?)

#### Next step
1. Try PCA on features_matthew_65brands and try to run again.
2. Further tune NN, RF.
3. Try Gradient Boosting.
4. Ensemble these models.

#### Data and models
1. Features: features_matthew_2, pc_diagnosis, features_matthew_65brands(useless)
2. Labels: response_variable_v2
3. Models: logistic regression, RF, Neural Network, KNN(useless)

#### Structure of code
1. Process input and output
2. Models

#### Issue to be discussed
1. In hold_out set, there are [16] ppl has no supply_day. How should we treet them?
2. In train data, we have 129 problematic ppl: 
3. 18 of them have no supply_day data when day >= 0
4. 34 of them have no supply_day data through out all period
5. 68 of them have no supply_day data when day > 0
6. 9 of them have no diagnosis data (but have supply_day data)

#### Others
1. Make sure make probability prediction when calculating roc_auc_score.
2. Ref: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html


## Process input and outpur

In [2]:
%%time
df = pd.read_csv('C:/Users/spong/Desktop/Humana/HMAHCC_COMP.csv')

rx_paid = df[df['event_descr']=='RX Claim - Paid']

rx_paid = rx_paid.drop(['event_attr2',
                        'event_attr7'],
                         axis = 1)

rx_paid.columns = ['id',
                   'event_descr',
                   'gpi_drug_class_description',
                   'rx_cost',
                   'net_paid_amount',
                   'brand_name',
                   'drug_group_description',
                   'generic_name',
                   'member_responsible_amount',
                   'gpi_drug_group8_id',
                   'Days',
                   'PAY_DAY_SUPPLY_CNT',
                   'PAYABLE_QTY',
                   'MME',
                   'DRUG_TYPE',
                   'Specialty',
                   'Specialty2',
                   'Specialty3']

opioid_col = ['id',
              'gpi_drug_class_description',
              'brand_name',
              'drug_group_description',
              'generic_name',
              'gpi_drug_group8_id',
              'PAY_DAY_SUPPLY_CNT',
              'PAYABLE_QTY',
              'MME',
              'Specialty',
              'Specialty2',
              'Specialty3',
              'Days']

opioid = rx_paid[(rx_paid['PAY_DAY_SUPPLY_CNT'].notnull())&(rx_paid['Days']<=0)][opioid_col]

<string>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 29.5 s


#### features_matthew_v2
supply_day, payable_qty, MME

In [2]:
features3 = pd.read_csv('features_matthew_v2.csv')
features3 = features3.set_index('key_0')
features_matthew_v2 = features3.copy()
features_matthew_v2.head()

,MME_on_day0,SUPPLY_CNT_on_day0,PAYABLE_QTY_on_day0,max_MME_prior,avg_MME_prior,total_SUPPLY_CNT_prior,total_PAYABLE_QTY_prior,supply_times
key_0,,,,,,,,
ID10010854159,15.0,5.0,15.0,0.0,0.000000,0.0,0.0,1
ID10013863216,10.0,90.0,180.0,0.0,0.000000,0.0,0.0,1
ID10024447278,50.0,3.0,20.0,22.5,20.000000,15.0,40.0,4
ID1002482139,60.0,30.0,120.0,90.0,37.254464,789.0,2895.0,29
ID1003386406,20.0,15.0,60.0,50.0,50.000000,3.0,20.0,2


In [113]:
# ######################################
# # code to generate features_matthew_v2
# ######################################

# opioid_grouped = opioid.groupby(by=['id'])
# idtestlist = opioid['id'].drop_duplicates()

# features3 = pd.DataFrame()
# for ID in idtestlist:
#     tmp = opioid_grouped.get_group(ID)

#     # MME (per day) on day 0
#     # Suuply_CNT on day 0
#     on_day0 = tmp[tmp['Days'] == 0] 
#     if not on_day0.empty:
#         MME0 = on_day0['MME'].values[0]
#         SC0 = on_day0['PAY_DAY_SUPPLY_CNT'].values[0]
#         PQ0 = on_day0['PAYABLE_QTY'].values[0]
#     else:
#         MME0 = 0
#         SC0 = 0
#         PQ0 = 0

#     # max MME (per day) prior to day 0
#     # average MME (per day) prior to day 0
#     # Total Supply_CNT prior to day 0
#     prior_day0 = tmp[tmp['Days'] < 0]
#     if not prior_day0.empty:
#         maxMME = np.nanmax(prior_day0['MME'].values)
#         avgMME = np.nanmean(prior_day0['MME'].values)
#         totalSC = np.sum(prior_day0['PAY_DAY_SUPPLY_CNT'].values)
#         totalPQ = np.sum(prior_day0['PAYABLE_QTY'].values)
#     else:
#         maxMME = 0
#         avgMME = 0
#         totalSC = 0
#         totalPQ = 0

#     output = pd.DataFrame({'MME_on_day0': MME0, 
#                          'SUPPLY_CNT_on_day0': SC0,
#                            'PAYABLE_QTY_on_day0': PQ0,
#                          'max_MME_prior': maxMME,
#                          'avg_MME_prior': avgMME,
#                          'total_SUPPLY_CNT_prior': totalSC,
#                           'total_PAYABLE_QTY_prior': totalPQ},
#                           index = [ID])

#     features3 = features3.append(output, sort=False)

# # MME_on_day0, max_MME_prior, avg_MME_prior has some missing value, fill with medians
# features3['MME_on_day0'] = features3['MME_on_day0'].fillna(np.nanmedian(features3['MME_on_day0']))
# features3['max_MME_prior'] = features3['max_MME_prior'].fillna(np.nanmedian(features3['max_MME_prior']))
# features3['avg_MME_prior'] = features3['avg_MME_prior'].fillna(np.nanmedian(features3['avg_MME_prior']))

# # add one more feature: supply_times
# supply_times = opioid.groupby(by=['id'])['PAY_DAY_SUPPLY_CNT'].count()
# supply_times = pd.DataFrame(supply_times)
# supply_times.columns = ['supply_times']
# features3 = features3.merge(supply_times, left_on=features3.index.values, right_on=supply_times.index.values)
# features3.to_csv('features_matthew_v2.csv')

#### features_matthew_65brands 
Entries are PAY_DAY_SUPPLY_CNT mean

This is not working, probably I can try PCA on this

In [4]:
# features = pd.read_csv('features_matthew_65brands.csv')
# features = features.set_index(['Unnamed: 0'])

# features_matthew_65brands = features.copy()
# features_matthew_65brands.head()

,TRAMADOL HCL,OXYCODONE/ACETAMINOPHEN,HYDROCODONE/ACETAMINOPHEN,OXYCODONE HCL,ACETAMINOPHEN/CODEINE,HYDROMORPHONE HCL,MORPHINE SULFATE ER,ACETAMINOPHEN/CODEINE PHO,ACETAMINOPHEN/CODEINE #3,FENTANYL,...,MEPERITAB,HYDROMORPHONE HCL ER,AVINZA,PERCOCET,NUCYNTA ER,EXALGO,VICOPROFEN,TREZIX,LORCET HD,LEVORPHANOL TARTRATE
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
ID10010854159,5.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID10013863216,90.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID10024447278,0.0,4.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID1002482139,0.0,0.0,28.178571,30.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID1003386406,15.0,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#############################################
## code to generate features_matthew_65brands
#############################################

# opioid2_grouped = opioid.groupby(by=['id'])

# idtestlist = opioid['id'].drop_duplicates()

# ## those commented-out codes are used to get other entry values
# # def product_sum(df):
# #     return(df['MME'].values.dot(df['PAY_DAY_SUPPLY_CNT']e.values))

# features = pd.DataFrame()
# for ID in idtestlist:
#     tmp = opioid2_grouped.get_group(ID)
#     output = pd.DataFrame(tmp.groupby(by='brand_name')['PAY_DAY_SUPPLY_CNT'].mean()).T
# #     output = output.iloc[0:1,:]
#     output.index = [ID]
#     # features = pd.concat([output, features], axis=1, sort=False)
#     features = features.append(output, sort=False)

# features = features.fillna(0)
# features.to_csv('features_matthew_65brands.csv')

Wall time: 3min 37s


#### pc_diagnosis

In [3]:
pc_diagnosis = pd.read_csv('pc_diagnosis.csv')
pc_diagnosis = pc_diagnosis.set_index('id')
pc_diagnosis.head()

,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6,PC 7,PC 8,PC 9,PC 10
id,,,,,,,,,,
ID10010854159,0.008840,0.006021,0.001393,0.003137,-0.000543,-0.000012,-0.002715,0.000189,-0.002999,0.000369
ID10013863216,0.004783,-0.004110,0.009211,0.006220,-0.000302,-0.001218,-0.004039,0.003508,-0.007012,0.007118
ID10024447278,0.007123,0.018349,-0.002873,0.001940,-0.000999,0.001292,0.000691,-0.000391,-0.000094,-0.002040
ID1002482139,-0.003584,-0.004210,0.000235,-0.003246,-0.001197,0.002330,0.000364,-0.001483,0.000771,-0.001723
ID1003386406,0.001259,-0.000543,-0.000644,-0.003560,-0.000509,-0.003400,-0.002201,-0.008570,0.005852,0.004095


#### Response variables

In [4]:
response_variable = pd.read_csv('response_variable_v2.csv')
response_variable = response_variable.dropna()
response_variable = response_variable.set_index('id')
response_variable.head()

,naive,LTOT_v2
id,,
ID10010854159,"[0, 706]",False
ID10013863216,[0],True
ID10024447278,[0],True
ID1002482139,[0],True
ID1003386406,[0],False


#### Merge all features and response variables

In [5]:
df2 = pd.concat([response_variable[['LTOT_v2']], features_matthew_v2, pc_diagnosis], 
                axis=1, join='inner')

df2.head()

,LTOT_v2,MME_on_day0,SUPPLY_CNT_on_day0,PAYABLE_QTY_on_day0,max_MME_prior,avg_MME_prior,total_SUPPLY_CNT_prior,total_PAYABLE_QTY_prior,supply_times,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6,PC 7,PC 8,PC 9,PC 10
ID10010854159,False,15.0,5.0,15.0,0.0,0.000000,0.0,0.0,1,0.008840,0.006021,0.001393,0.003137,-0.000543,-0.000012,-0.002715,0.000189,-0.002999,0.000369
ID10013863216,True,10.0,90.0,180.0,0.0,0.000000,0.0,0.0,1,0.004783,-0.004110,0.009211,0.006220,-0.000302,-0.001218,-0.004039,0.003508,-0.007012,0.007118
ID10024447278,True,50.0,3.0,20.0,22.5,20.000000,15.0,40.0,4,0.007123,0.018349,-0.002873,0.001940,-0.000999,0.001292,0.000691,-0.000391,-0.000094,-0.002040
ID1002482139,True,60.0,30.0,120.0,90.0,37.254464,789.0,2895.0,29,-0.003584,-0.004210,0.000235,-0.003246,-0.001197,0.002330,0.000364,-0.001483,0.000771,-0.001723
ID1003386406,False,20.0,15.0,60.0,50.0,50.000000,3.0,20.0,2,0.001259,-0.000543,-0.000644,-0.003560,-0.000509,-0.003400,-0.002201,-0.008570,0.005852,0.004095


In [6]:
# Base line
df2['LTOT_v2'].value_counts()[False]/sum(df2['LTOT_v2'].value_counts())

0.5047220820416697

In [25]:
X = df2.iloc[:,1:]
y = df2.iloc[:,0].map(lambda x: 1 if x == True else 0 )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

## Issue to be discussed: 16 ppl in holdout data has no supply_day values

In [8]:
# df_hould_out = pd.read_csv('C:/Users/spong/Desktop/Humana/HMAHCC_HOLDOUT.csv')
# opioid_hold_out = df_hould_out[df_hould_out['PAY_DAY_SUPPLY_CNT'].notnull()]
# len(opioid_hold_out['ID'].drop_duplicates())

5984

## Models

### Logistics

In [35]:
%%time

logit_model = LogisticRegression()

cv_results = cross_validate(logit_model, X_train, y_train, cv=3,
                            scoring= 'roc_auc',
                            return_train_score=True)

print('train roc_auc: ', np.mean(cv_results['test_score']))

logit_model.fit(X_train, y_train)
prediction_on_X_test = logit_model.predict_proba(X_test)
print('test roc_auc: ', roc_auc_score(y_test, prediction_on_X_test[:,1]))

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


train roc_auc:  0.8607554404440675
test roc_auc:  0.8680379846492545
Wall time: 261 ms


### RF tuned on max_depth

In [10]:
%%time
parameters = {'max_depth': list(range(1,31,1)),
             'n_estimators': [100, 300]}
rf = RandomForestClassifier()
rf_grid = GridSearchCV(rf, parameters, scoring='roc_auc', cv=5)
rf_grid.fit(X_train, y_train)

rf_grid.best_params_

Wall time: 24min 55s


{'max_depth': 9, 'n_estimators': 300}

In [37]:
%%time
rf_model = RandomForestClassifier(n_estimators=rf_grid.best_params_['n_estimators'], 
                                  max_depth=rf_grid.best_params_['max_depth'],
                                  random_state=100)

cv_results = cross_validate(rf_model, X_train, y_train, cv=3,
                            scoring= 'roc_auc',
                            return_train_score=True,
                            return_estimator =True)

print('train roc_auc: ', np.mean(cv_results['test_score']))

rf_model.fit(X_train, y_train)
prediction_on_X_test = rf_model.predict_proba(X_test)
print('test roc_auc: ', roc_auc_score(y_test, prediction_on_X_test[:,1]))

train roc_auc:  0.883797294400928
test roc_auc:  0.8929639077123037
Wall time: 12.6 s


In [ ]:
# parameters we can use to tune RF
# parameters = {'n_estimators':list(range(100,2100,100)),
#               'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4],
#               'max_features': ['auto', 'sqrt']}

#### RF features importance

In [12]:
for idx,estimator in enumerate(cv_results['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print('===================================')
    print(feature_importances)

Features sorted by their score for estimator 0:
                         importance
SUPPLY_CNT_on_day0         0.249045
PAYABLE_QTY_on_day0        0.129916
total_SUPPLY_CNT_prior     0.102767
total_PAYABLE_QTY_prior    0.093780
supply_times               0.055516
PC 5                       0.048653
PC 6                       0.044982
MME_on_day0                0.034482
avg_MME_prior              0.031437
PC 3                       0.026181
PC 7                       0.025797
max_MME_prior              0.024172
PC 4                       0.024128
PC 10                      0.023245
PC 9                       0.022927
PC 8                       0.022111
PC 2                       0.021673
PC 1                       0.019190
Features sorted by their score for estimator 1:
                         importance
SUPPLY_CNT_on_day0         0.264216
PAYABLE_QTY_on_day0        0.116777
total_SUPPLY_CNT_prior     0.108109
total_PAYABLE_QTY_prior    0.090165
supply_times               0.055570
PC 5

#### Code to visualize the effect of different hyperparameters of RF

In [64]:
# grid = list(range(1,31,1))
# train_accuracy = list()
# test_accuracy = list()
# for i in grid:
#     rf_model = RandomForestClassifier(n_estimators=100, max_depth=i,
#                                       random_state=0)
#     cv_results = cross_validate(rf_model, X_train, y_train, cv=3,
#                             scoring= 'accuracy',
#                             return_train_score=True)

#     train_accuracy.append(np.mean(cv_results['train_score']))
#     test_accuracy.append(np.mean(cv_results['test_score']))

# test_res = pd.DataFrame([train_accuracy, test_accuracy], index = ['train_accuracy', 'test_accuracy']).T
# test_res.set_index = grid
# test_res.plot()
# plt.ylabel('Accuracy')
# plt.xlabel('max_depth')

Wall time: 33.9 s


### KNN classifier on tuned K 

In [80]:
# %%time
# parameters = {'n_neighbors': list(range(5,205,10))}
# neigh = KNeighborsClassifier()
# neigh_grid = GridSearchCV(neigh, parameters, scoring='roc_auc', cv=5)
# neigh_grid.fit(X_train, y_train)

# neigh_grid.best_params_

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Wall time: 56.1 s


{'n_neighbors': 25}

In [152]:
# %%time
# neigh = KNeighborsClassifier(n_neighbors=25)

# cv_results = cross_validate(neigh, X_train, y_train, cv=3,
#                             scoring= 'roc_auc',
#                             return_train_score=True,
#                            return_estimator =True)

# print('train roc_auc: ', np.mean(cv_results['test_score']))

# neigh.fit(X_train, y_train)
# prediction_on_x_test = neigh.predict(X_test)
# print('test roc_auc: ', roc_auc_score(y_test, prediction_on_x_test))

test roc_auc:  0.8159436471332583
Wall time: 8.43 s


### Neural Network

In [13]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [14]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scale = scaler.transform(X)
y_array = y.values
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_array, test_size=0.33, random_state=33)
X_train_mlp, X_validation, y_train_mlp, y_validation = train_test_split(X_train, y_train, test_size=0.15, random_state=33)

In [21]:
%%time
model = Sequential()

#First Hidden Layer
model.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=X_train.shape[1]))

#Second  Hidden Layer
model.add(Dense(64, activation='relu', kernel_initializer='random_normal'))

#Output Layer
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# there's no easy way to incorporate roc_auc in MLP's training process
# so I use accuracy, which can be a proxy of roc_auc, to train models
# then use roc_auc to assess the prediction on test set

# add early stop to prevent overfitting
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train, y_train,
                    validation_split = 0.15,
                    epochs=4000, verbose=0, callbacks=[es, mc])

saved_model = load_model('best_model2.h5')

_, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_acc = saved_model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Epoch 00001: val_loss improved from inf to 0.42823, saving model to best_model2.h5

Epoch 00002: val_loss improved from 0.42823 to 0.42290, saving model to best_model2.h5

Epoch 00003: val_loss improved from 0.42290 to 0.41398, saving model to best_model2.h5

Epoch 00004: val_loss did not improve from 0.41398

Epoch 00005: val_loss improved from 0.41398 to 0.41362, saving model to best_model2.h5

Epoch 00006: val_loss improved from 0.41362 to 0.41234, saving model to best_model2.h5

Epoch 00007: val_loss improved from 0.41234 to 0.40958, saving model to best_model2.h5

Epoch 00008: val_loss did not improve from 0.40958

Epoch 00009: val_loss improved from 0.40958 to 0.40749, saving model to best_model2.h5

Epoch 00010: val_loss improved from 0.40749 to 0.40671, saving model to best_model2.h5

Epoch 00011: val_loss improved from 0.40671 to 0.40517, saving model to best_model2.h5

Epoch 00012: val_loss improved from 0.40517 to 0.40494, saving model to best_model2.h5

Epoch 00013: val_lo

In [23]:
prediction_on_X_train = saved_model.predict_proba(X_train)
prediction_on_X_train = [x[0] for x in prediction_on_X_train]
print('roc_auc on train: ', roc_auc_score(y_train, prediction_on_X_train))

prediction_on_X_test = model.predict_proba(X_test)
prediction_on_X_test = [x[0] for x in prediction_on_X_test]
print('roc_auc on test: ', roc_auc_score(y_test, prediction_on_X_test))

roc_auc on train:  0.8965876529139651
roc_auc on test:  0.8965608261117267


### LightGBM

In [ ]:
# import lightgbm as lgb
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

In [303]:
# train_data = lgb.Dataset(X_train.values, y_train.values,
#                          feature_name = list(X.columns))

# param = {'num_leaves': 31, 'objective': 'binary'}
# param['metric'] = 'auc'

# num_round = 1000
# bst = lgb.train(param, train_data, num_round)

# y_test_predict = pd.Series(bst.predict(X_test)).map(lambda x: 1 if x >0.5 else 0)
# accuracy_score(y_test, y_test_predict)